## MISA (2024-2025)
- Alohan'ny mamerina dia avereno atao Run ny notebook iray manontolo. Ny fanaovana azy dia redémarrena mihitsy ny kernel aloha (jereo menubar, safidio **Kernel$\rightarrow$Restart Kernel and Run All Cells**).

- Izay misy hoe `YOUR CODE HERE` na `YOUR ANSWER HERE` ihany no fenoina. Afaka manampy cells vaovao raha ilaina. Aza adino ny mameno references eo ambany raha ilaina.

## References
* [PCA avec SVD](https://www.google.com/search?q=pca+avec+svd&oq=pca+avec+svd&gs_lcrp=EgZjaHJvbWUyBggAEEUYOTIHCAEQIRigATIHCAIQIRigATIHCAMQIRifBTIHCAQQIRifBTIHCAUQIRifBTIHCAYQIRifBTIHCAcQIRifBTIHCAgQIRifBTIHCAkQIRifBdIBCDUwMjBqMGo3qAIAsAIA&sourceid=chrome&ie=UTF-8#fpstate=ive&vld=cid:c7c3b00d,vid:ILu4-Lk-gZQ,st:0)
* [cvxpy](https://www.cvxpy.org/)

---

In [181]:
from sklearn.decomposition  import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing, load_iris, load_breast_cancer, make_blobs
import numpy as np
from random import randrange
from sklearn.metrics import accuracy_score
import cvxpy as cp

def grad_check_sparse(f, x, analytic_grad, num_checks=12, h=1e-5, error=1e-9):
    """
    sample a few random elements and only return numerical
    in this dimensions
    """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (
            abs(grad_numerical) + abs(grad_analytic)
        )
        print(
            "numerical: %f analytic: %f, relative error: %e"
            % (grad_numerical, grad_analytic, rel_error)
        )
        assert rel_error < error

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

data = load_iris()
X, y = data.data, data.target

# PCA

#### pour la recherche des vecteurs propres on peut utiliser le SVD

In [182]:
class PrincipalComponentAnalysis:
    def __init__(self, n_components):
        self.n_components = n_components
        self.components = None
    
    def fit(self, X):
        # Centrer les données
        X_centered = X - np.mean(X, axis=0)
        
        # Calculer la SVD
        U, S, Vt = np.linalg.svd(X_centered, full_matrices=False)
        
        # Les composantes principales sont dans Vt
        self.components = Vt[:self.n_components]
    
    def transform(self, X):
        # Projeter les données sur les composantes principales
        X_centered = X - np.mean(X, axis=0)
        return np.dot(X_centered, self.components.T)


In [183]:
X_centered = X - np.mean(X, axis=0)

pca = PCA(n_components=3)
pca.fit(X_centered)
X_pca_trans = pca.transform(X_centered)

model = PrincipalComponentAnalysis(n_components=3)
model.fit(X_centered)
X_model_trans = model.transform(X_centered)

sign_correct_X_model_trans = np.concatenate([np.expand_dims(X_model_trans[:,0],axis=1),-X_model_trans[:,1:]],axis=1)

error = rel_error(X_pca_trans, sign_correct_X_model_trans)
print(error)
assert  error < 1e-11

6.99364782835624e-14


# Binary Linear SVM with CVXPY

## Hard margin 

In [184]:
X2, y2 = make_blobs(n_samples=300, centers=2, n_features=12, random_state=47)
scaler = StandardScaler()
X2 = scaler.fit_transform(X2)
y2[y2 == 0] = -1

$$\min_{\mathbf{w},b}\frac{1}{2}||\mathbf{w}||^2$$ <br>
$$\text{s.t } y_i(\mathbf{w}^{\top}\mathbf{x}_i + b) \ge 1, \ i=1...N$$

In [185]:
import cvxpy as cp
import numpy as np

class LinearSVMHardMargin:
    def __init__(self):
        self.w = None  # Le vecteur normal à l'hyperplan
        self.b = 0  # Le biais de l'hyperplan
    
    def fit(self, X, y):
        # X : les données d'entrée (n_samples, n_features)
        # y : les labels (n_samples,)
        
        n_samples, n_features = X.shape
        
        # Définir les variables du problème d'optimisation
        w = cp.Variable(n_features)
        b = cp.Variable()
        
        # Définir les contraintes : y_i * (w^T x_i + b) >= 1
        constraints = [y[i] * (X[i] @ w + b) >= 1 for i in range(n_samples)]
        
        # Définir l'objectif : minimiser (1/2) * ||w||^2
        objective = cp.Minimize(0.5 * cp.sum_squares(w))  # Utiliser sum_squares pour calculer la norme

        # Définir et résoudre le problème d'optimisation
        problem = cp.Problem(objective, constraints)
        problem.solve()
        
        # Stocker les résultats dans les attributs de la classe
        self.w = w.value
        self.b = b.value
    
    def predict(self, X):
        """Retourne les étiquettes prédites (1 ou -1)"""
        # Prédire en fonction du signe de (X @ w + b)
        return np.sign(X @ self.w + self.b)


In [186]:
model = LinearSVMHardMargin()
model.fit(X2, y2)
pred = model.predict(X2)
accuracy = accuracy_score(y2, pred)
print(accuracy)
assert accuracy == 1

1.0


## Soft Margin

In [187]:
data3 = load_breast_cancer()
X3, y3 = data3.data, data3.target
scaler = StandardScaler()
X3 = scaler.fit_transform(X3)
y3[y3 == 0] = -1

$$L(\mathbf{w},b) = \frac{1}{N} \sum_{i=1}^N \max(0, y_i(\mathbf{w}^{\top}\mathbf{x}_i + b)) + \lambda||\mathbf{w}||^2_2$$

In [188]:
import cvxpy as cp
import numpy as np

class LinearSVMSoftMargin:
    def __init__(self, alpha=0.1):
        self.w = None  # Vecteur normal à l'hyperplan
        self.b = 0     # Biais de l'hyperplan
        self.alpha = alpha  # Paramètre C, qui est le poids de la pénalité pour les erreurs
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        
        # Définir les variables de décision
        w = cp.Variable(n_features)
        b = cp.Variable()
        
        # Calculer la perte Hinge
        hinge_loss = cp.maximum(0, 1 - cp.multiply(y, X @ w + b))
        
        # Définir l'objectif : minimiser la moyenne de la perte Hinge + régularisation
        objective = cp.Minimize(
            cp.mean(hinge_loss) + self.alpha * cp.sum_squares(w)
        )
        
        # Définir et résoudre le problème d'optimisation
        problem = cp.Problem(objective)
        problem.solve()
        
        # Stocker les résultats dans les attributs de la classe
        self.w = w.value
        self.b = b.value
    
    def predict(self, X):
        """Retourne les étiquettes prédites (1 ou -1)"""
        # Prédire en fonction du signe de (X @ w + b)
        return np.sign(X @ self.w + self.b)


In [189]:
model = LinearSVMSoftMargin(alpha=1e-3)
model.fit(X3, y3)
pred = model.predict(X3)
accuracy = accuracy_score(y3, pred)
print(accuracy)
assert accuracy >= 0.98

0.9876977152899824


# Multiclass Linear SVM

## Loss

$$L(\mathbf{W}) = \sum_{i=1}^N \sum_{j \neq y_i} \max(0, s_j - s_{y_i} + 1) + \lambda||\mathbf{w}||^2_2$$ <br>
$$\text{where } s_j = (f(\mathbf{x}_i;\mathbf{W}))_j = (\mathbf{W}\mathbf{x}_i)_j \text{ is the score for the j-th class}$$

In [190]:
W = np.random.randn(X.shape[1], 3) * 0.0001

In [191]:
def svm_loss_naive(W, X, y, alpha):
    """
    Fonction de perte multiclasses SVM avec boucles for

    Inputs:
    - W: tableau de forme (D, C) contenant les poids
    - X: tableau de forme (N, D) contenant un lot de données
    - y: tableau de forme (N,) contenant les étiquettes des données
    - alpha: (float) régularisation des poids

    Returns:
    - loss: une seule valeur, la perte totale
    - dW: gradient par rapport aux poids W; même forme que W
    """
    
    # Initialisation de la perte et du gradient
    loss = 0.0
    dW = np.zeros_like(W)
    N = X.shape[0]  # Nombre d'exemples
    C = W.shape[1]  # Nombre de classes
    
    # Parcourir tous les exemples
    for i in range(N):
        scores = X[i].dot(W)  # Calcul des scores pour chaque classe
        correct_class_score = scores[y[i]]  # Score de la classe correcte
        
        # Parcourir toutes les classes
        for j in range(C):
            if j == y[i]:
                continue  # On saute la classe correcte

            margin = scores[j] - correct_class_score + 1  # Calcul de la perte de marge Hinge

            # Si la marge est positive, ajouter à la perte
            if margin > 0:
                loss += margin
                # Calcul du gradient
                dW[:, j] += X[i]  # Poids de la classe j
                dW[:, y[i]] -= X[i]  # Poids de la classe correcte

    # Moyenne de la perte sur tous les exemples
    loss /= N
    
    # Ajouter la régularisation (penalisation des grands poids)
    loss += 0.5 * alpha * np.sum(W * W)  # np.sum(W * W) calcule ||W||^2

    # Moyenne du gradient sur tous les exemples
    dW /= N
    
    # Ajouter la régularisation au gradient
    dW += alpha * W  # Gradient de la régularisation

    return loss, dW


In [192]:
# NO REGLARIZATION
loss, dW = svm_loss_naive(W, X, y, 0.0)

f = lambda W: svm_loss_naive(W, X, y, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

numerical: 2.296000 analytic: 2.296000, relative error: 2.772656e-12
numerical: 0.953333 analytic: 0.953333, relative error: 6.972861e-12
numerical: -0.502000 analytic: -0.502000, relative error: 8.843524e-12
numerical: -0.370667 analytic: -0.370667, relative error: 6.389224e-12
numerical: -0.126667 analytic: -0.126667, relative error: 3.503670e-10
numerical: 2.296000 analytic: 2.296000, relative error: 2.772656e-12
numerical: -1.794000 analytic: -1.794000, relative error: 1.653576e-13
numerical: 0.083333 analytic: 0.083333, relative error: 6.334458e-11
numerical: -1.794000 analytic: -1.794000, relative error: 1.653576e-13
numerical: -0.502000 analytic: -0.502000, relative error: 8.843524e-12
numerical: -0.092667 analytic: -0.092667, relative error: 8.346823e-11
numerical: -1.794000 analytic: -1.794000, relative error: 1.653576e-13


In [193]:
#REGLARIZATION
loss, dW = svm_loss_naive(W, X, y, 2)

f = lambda W: svm_loss_naive(W, X, y, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

numerical: 0.287405 analytic: 0.287405, relative error: 7.303629e-12
numerical: -1.793824 analytic: -1.793824, relative error: 1.125062e-12
numerical: -0.826779 analytic: -0.826779, relative error: 3.517214e-11
numerical: -0.744808 analytic: -0.744808, relative error: 6.420085e-12
numerical: -0.126603 analytic: -0.126603, relative error: 4.089537e-10
numerical: -1.793824 analytic: -1.793824, relative error: 1.125062e-12
numerical: 0.287405 analytic: 0.287405, relative error: 7.303629e-12
numerical: 0.953445 analytic: 0.953445, relative error: 4.232425e-12
numerical: 0.083117 analytic: 0.083117, relative error: 4.548635e-11
numerical: -0.126603 analytic: -0.126603, relative error: 4.089537e-10
numerical: -0.370497 analytic: -0.370497, relative error: 8.241184e-12
numerical: 0.953445 analytic: 0.953445, relative error: 4.232425e-12


In [205]:
import numpy as np

def svm_loss_vectorized(W, X, y, alpha):
    """
    Multiclass SVM loss function WITHOUT FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization 

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W; same shape as W
    """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)
    
    N, D = X.shape
    C = W.shape[1]
    
    # Compute the scores (N, C)
    S = X @ W
    
    # Indices for the correct classes
    idx = np.arange(N)
    
    # Calculate the margins (N, C)
    margins = np.maximum(0, S.T - S[idx, y] + 1).T
    margins[idx, y] = 0  # Set the margin for the correct class to 0
    
    # Create a matrix where margin > 0 is 1, else 0 (Violation of margin)
    margin_violations = np.where(margins > 0, 1, 0)
    
    # Create a matrix to store the sum of violations for the correct class
    correct_class_violations = np.zeros_like(margin_violations)
    correct_class_violations[idx, y] = np.sum(margin_violations, axis=1)
    
    # Compute the gradient (D, C)
    dW += X.T @ margin_violations
    dW -= X.T @ correct_class_violations
    
    # Calculate the loss (average margin + regularization)
    loss = np.sum(margins) # Average margin across all examples
    loss += 0.5 * alpha * np.sum(W**2)  # Add regularization term
    
    # Add regularization gradient
    dW += alpha * W
    
    return loss, dW


In [206]:
# NO REGLARIZATION
loss, dW = svm_loss_vectorized(W, X, y, 0.0)

f = lambda W: svm_loss_vectorized(W, X, y, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

numerical: 125.600000 analytic: 125.600000, relative error: 4.831860e-12
numerical: -269.100000 analytic: -269.100000, relative error: 5.033315e-12
numerical: 143.000000 analytic: 143.000000, relative error: 9.641717e-12
numerical: -55.600000 analytic: -55.600000, relative error: 3.554202e-11
numerical: -111.700000 analytic: -111.700000, relative error: 2.865135e-12
numerical: -75.300000 analytic: -75.300000, relative error: 1.680467e-11
numerical: -111.700000 analytic: -111.700000, relative error: 2.865135e-12
numerical: -269.100000 analytic: -269.100000, relative error: 5.033315e-12
numerical: 12.500000 analytic: 12.500000, relative error: 1.255126e-10
numerical: -19.000000 analytic: -19.000000, relative error: 6.567547e-11
numerical: -55.600000 analytic: -55.600000, relative error: 3.554202e-11
numerical: -269.100000 analytic: -269.100000, relative error: 5.033315e-12


In [207]:
# REGLARIZATION
loss, dW = svm_loss_vectorized(W, X, y, 2)

f = lambda W: svm_loss_vectorized(W, X, y, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

numerical: -124.000112 analytic: -124.000112, relative error: 6.664713e-12
numerical: -111.700141 analytic: -111.700141, relative error: 1.813249e-12
numerical: -269.099824 analytic: -269.099824, relative error: 7.956183e-12
numerical: -18.999936 analytic: -18.999936, relative error: 4.425060e-11
numerical: 43.100072 analytic: 43.100072, relative error: 4.169692e-12
numerical: 43.100072 analytic: 43.100072, relative error: 4.169692e-12
numerical: 12.499784 analytic: 12.499784, relative error: 3.479915e-11
numerical: -75.300471 analytic: -75.300471, relative error: 2.798417e-11
numerical: -13.900196 analytic: -13.900196, relative error: 8.172583e-11
numerical: 12.499784 analytic: 12.499784, relative error: 3.479915e-11
numerical: 143.000111 analytic: 143.000111, relative error: 1.594864e-11
numerical: -13.900196 analytic: -13.900196, relative error: 8.172583e-11


## Gradient descent

In [ ]:
import numpy as np

class LinearModel():
    def __init__(self, fit_intercept=True):
        self.W = None
        self.fit_intercept = fit_intercept

    def train(self, X, y, learning_rate=1e-3, alpha=0, num_iters=100, batch_size=200, verbose=False):
        if self.fit_intercept:
            X = np.hstack((np.ones((X.shape[0], 1)), X))  # Ajouter une colonne de 1 pour l'interception

        N, d = X.shape
        C = np.max(y) + 1  # Nombre de classes
        if self.W is None:  # Initialisation des poids
            self.W = 0.001 * np.random.randn(d, C)  # Initialisation aléatoire des poids

        # Exécution de la descente de gradient stochastique pour optimiser W
        loss_history = []
        for it in range(num_iters):
            # Sélectionner un mini-batch
            indices = np.random.choice(N, batch_size, replace=False)  # Échantillonner batch_size éléments
            X_batch = X[indices, :]  # Récupérer les données du mini-batch
            y_batch = y[indices]  # Récupérer les labels du mini-batch

            # Évaluer la perte et le gradient
            loss, dW = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            # Mise à jour des poids
            self.W -= learning_rate * dW  # Mettre à jour les poids avec la descente de gradient

            if verbose and it % 1000 == 0:
                print(f"iteration {it} / {num_iters}: loss {loss}")

        return loss_history

    def predict(self, X):
        pass

    def loss(self, X_batch, y_batch, reg):
        pass


class LinearSVM(LinearModel):
    """ Modèle SVM linéaire """

    def loss(self, X_batch, y_batch, alpha):
        return svm_loss_vectorized(self.W, X_batch, y_batch, alpha)
    
    def predict(self, X):
        """ 
        Effectue la prédiction avec les poids actuels

        Inputs:
        - X: array de forme (N, D) contenant les données d'entrée

        Returns:
        - y_pred: array unidimensionnel de taille N, chaque élément représentant la classe prédite
        """
        if self.fit_intercept:
            X = np.hstack((np.ones((X.shape[0], 1)), X))  # Ajouter une colonne de 1 pour l'interception

        scores = X @ self.W  # Calculer les scores en fonction des poids
        y_pred = np.argmax(scores, axis=1)  # Prendre la classe ayant le score le plus élevé pour chaque exemple

        return y_pred


In [198]:
model = LinearSVM(fit_intercept=True)
model.train(X, y, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X)
model_accuracy = accuracy_score(y, pred)
print(model_accuracy)
assert model_accuracy > 0.97

iteration 0 / 75000: loss 127.65063841826927
iteration 1000 / 75000: loss 2.4469208776773517
iteration 2000 / 75000: loss 5.84046734572323
iteration 3000 / 75000: loss 7.671618959892193
iteration 4000 / 75000: loss 2.779870180175286
iteration 5000 / 75000: loss 4.508324116190355
iteration 6000 / 75000: loss 6.243137024106769
iteration 7000 / 75000: loss 5.091934392656745
iteration 8000 / 75000: loss 3.021617383523456
iteration 9000 / 75000: loss 1.6459575790330283
iteration 10000 / 75000: loss 4.241049698323684
iteration 11000 / 75000: loss 2.957367559972961
iteration 12000 / 75000: loss 6.957825083600356
iteration 13000 / 75000: loss 7.869371554142699
iteration 14000 / 75000: loss 2.287048813675976
iteration 15000 / 75000: loss 5.520541069981035
iteration 16000 / 75000: loss 3.3314935320971255
iteration 17000 / 75000: loss 0.4835014268790623
iteration 18000 / 75000: loss 4.625005225091561
iteration 19000 / 75000: loss 1.6211272526550842
iteration 20000 / 75000: loss 2.8649155807829
it